In [1]:
import requests as req
from bs4 import BeautifulSoup as bs
import sys
import urllib.request
from urllib.parse import quote
import pandas as pd

In [9]:
# 전체 코드

## RISS에서 크롤링하고자 하는 검색어 입력 -> URL

page = int(input("페이지 수 : "))
search = input("검색어 : ")

def URL(search, page_num):
    URL = "https://www.riss.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&query="+str(search) \
    +"&queryText=&iStartCount="+str(page_num)+"&iGroupView=5&icate=all&colName=re_a_kor&exQuery=&exQueryText=&order=%2FDESC&onHanja=false&strSort=RANK&pageScale=10&orderBy=&fsearchMethod=search&isFDetailSearch=N&sflag=1&searchQuery=" \
    +str(search)+"&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&resultKeyword=" \
    +str(search)+"&pageNumber="+str(page_num)+"&p_year1=&p_year2=&dorg_storage=&mat_type=&mat_subtype=&fulltext_kind=&t_gubun=&learning_type=&language_code=&ccl_code=&language=&inside_outside=&fric_yn=&image_yn=&regnm=&gubun=&kdc=&ttsUseYn="
    return URL

## 새로운 데이터프레임 만들기

df_info = pd.DataFrame(columns = ['저자', '국문 제목', '수록지','연도', '국문 요약', '링크'])

authors = []
k_titles = []
books = []
year = []
abstracts = []

links = []

headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36'}

for i in range(page) :
    page_num = i * 10
    # print(page_num)
    url = URL(search, page_num)
    res = req.get(url.format(search, page_num, search, search), headers = headers)
    html = bs(res.text, 'html.parser')
    items = html.findAll('div', {'class' : 'cont'})

    
    for item in items :
        # print(item)
        r_items = item.findAll('p', {'class' : 'title'})
        
        try : 
            abstract = item.find("p", {'class' : 'preAbstract'}).text.strip()
            abstracts.append(abstract)
        except Exception as ex :
            abstracts.append("")
        # print(abstract)
        # print("************")
    
        for detail in r_items :
            links.append("http://www.riss.kr"+detail.find("a")["href"])
            k_titles.append(detail.find("a").text)
        
            
    for item in items :
        r_items = item.findAll('p', {'class' : 'etc'})
        for detail in r_items :
            authors.append(item.find("span", {'class' : 'writer'}).text)
            books.append(item.find("span", {'class' : 'assigned'}).text)
            year.append(item.find("span", {'class' : ''}).text)
          
    #print(len(links), len(authors), len(k_titles), len(books), len(year), len(abstracts))

df_info['저자'] = authors
df_info['국문 제목'] = k_titles
df_info['수록지'] = books
df_info['연도'] = year
df_info['국문 요약'] = abstracts
df_info['링크'] = links

#with open('./paper-a.txt', 'w', encoding = 'UTF-8') as f :
 #   for abstract in abstracts :
  #      f.write(abstract + '\n'+'\n'+"end"+'\n'+'\n')

페이지 수 : 1
검색어 : 인체치수
10 10 10 10 10 10


In [11]:
df_info.to_csv("./paper-a.csv", na_rep='NaN', encoding="utf-8-sig")